pickle의 모듈의 직렬화 방식은 안전하지 않다

pickle의 데이터가 자신을 역직렬화 하는 파이썬 프로ㅡ램의 일부를 취약하게 만들 수도 있다.

반대로 json 모듈은 설계상 안전하다. 직렬화한 Json 데이터에는 객체 계층 구조를 간단하게 묘사한 값이 들어있다.

JSON 데이터를 역직렬화해도 파이썬 프로그램이 추가적인 위험에 노출되는 일은 없다.
서로를 신뢰할 수 없는 프로그램이 통신해야 할 경우에는 JSON 같은 형식을 사용해야한다.



In [1]:
class GameState:
    def __init__(self):
        self.level = 0
        self.lives = 4
        
state = GameState()
state.level += 1 # 레벨을 깼다
state.lives -= 1 # 한 번 재시도 해야함

print(state.__dict__)

{'level': 1, 'lives': 3}


In [2]:
# 사용자가 게임을 그만두면 나중에 이어서 저장할 수 있도록 프로그램이 게임 상태를 파일에 저장하며

# pickle 모듈을 사용하면 사앹를 수비게 저장할 수 있다

import pickle

state_path = 'game_state.bin'
with open(state_path, 'wb') as f:
    pickle.dump(state, f)



In [3]:
# 나중에 이 파일에 대해 load 함수를 호출하면 직렬화 한 적이 전혀 없었던 것 처럼 다시 GameState 객체를 돌려 받을 수 있다.

with open(state_path, 'rb') as f:
    state_after = pickle.load(f)
    
print(state_after.__dict__) # 원본 dict 그대로

{'level': 1, 'lives': 3}


이런 접근 방법을 사용하면 시간이 지나면서 게임 기능이 확장 될 때 문제가 발생한다.

플레이어가 최고점을 목표로 점수를 얻을 수 있게 게임을 변경하면



In [4]:
class GameState:
    def __init__(self):
        self.level = 0
        self.lives = 4
        self.points = 0 # 새로운 필드
        

In [5]:
state = GameState()
serialized = pickle.dumps(state)
state_after = pickle.loads(serialized)
print(state_after.__dict__)



{'level': 0, 'lives': 4, 'points': 0}


In [7]:
with open(state_path, 'rb') as f:
    state_after = pickle.load(f)
    
print(state_after.__dict__) # points필드가 사라졌다
print(type(state_after)) # GameState의 Instance임에도 불구하고!
assert isinstance(state_after, GameState)

{'level': 1, 'lives': 3}
<class '__main__.GameState'>


디폴트 애트리뷰트 값 가장 간단한 경우, 디폴트 인자를 생성자로 사용하면 GameState객체를 언피클 했을때도 항상 필요한 모든 인자를 포함시킬 수 이다



In [15]:
import copyreg

class GameState:
    def __init__(self, level= 0, lives = 4, points = 0):
        self.level = level
        self.lives = lives
        self.points = points
        
def unpickle_game_state(kwargs):
    return GameState(**kwargs)

def pickle_game_state(game_state):
    kwargs = game_state.__dict__
    return unpickle_game_state, (kwargs, )

copyreg.pickle(GameState, pickle_game_state)

In [16]:
state = GameState()
state.points += 100
serialized = pickle.dumps(state)
state_after = pickle.loads(serialized)
print(state_after.__dict__)

{'level': 0, 'lives': 4, 'points': 100}


In [11]:
class GameState:
    def __init__(self, level= 0, lives = 4, points = 0, magic=5):
        self.level = level
        self.lives = lives
        self.points = points
        self.magic = magic # 추가한 필드

In [12]:
print('이전', state.__dict__)
state_after = pickle.loads(serialized)
print('이후:', state_after.__dict__)

이전 {'level': 0, 'lives': 4, 'points': 100}
이후: {'level': 0, 'lives': 4, 'points': 100, 'magic': 5}


파이썬 객체의 필드를 제거해 예전 버전 객체와의 하위호환성이 없어지는 경우도 발생한다..

이런 식의 변경이 일어나면 디폴트 인자를 사용하는 접근 방법을 사용할 수 없다.

In [25]:
class GameState:
    def __init__(self, level=0, points=0, magic=5):
        self.level = level
        self.points = points
        self.magic = magic

pickle.loads(serialized) # 이전 데이터의 모든 필드가 GameState 생성자에게 전잘되므로 클래스에서 제거된 필드도 생성자에게 전달된다

In [26]:
def pickle_game_state(game_state):
    kwargs = game_state.__dict__
    kwargs['version'] = 2 # 버전 추가추가
    return unpickle_game_state, (kwargs, ) 

# 이전 버전 데이터에는 version 인자가 들어 있지 않다. 


def unpickle_game_state(kwargs):
    version = kwargs.pop('version', 1) # 'version'이 있으면 value return 없으면 1 리턴
    if version == 1:
        del kwargs['lives']
        
    return GameState(**kwargs)

    

In [27]:
copyreg.pickle(GameState, pickle_game_state)
print('이전: ', state.__dict__)
state_after = pickle.loads(serialized)
print('이후:', state_after.__dict__)

이전:  {'level': 0, 'lives': 4, 'points': 100}
이후: {'level': 0, 'points': 100, 'magic': 5}


pickle을 할 때 마주칠 수 있는 다른 문제점으로 클래스 이름이 바뀌어 코드가 깨지는 경우가 생긴다

클래스 이름을 바꿀경우 무슨 일이 일어나는가?


In [29]:
del GameState # 기존 GameState 객체 완전 제거

In [30]:
class BetterGameState:
    def __init__(self, level=0, points=0, magic=5):
        self.level = level
        self.points = points
        self.magic = magic
        
pickle.loads(serialized)

NameError: name 'GameState' is not defined

In [31]:
print(serialized)
# 피클된 데이터 안에 클래스 임포트 경로가 들어있다

# 이 경우에도 copyreg이 해결방법이 될 수 있다.

b'\x80\x04\x95K\x00\x00\x00\x00\x00\x00\x00\x8c\x08__main__\x94\x8c\x13unpickle_game_state\x94\x93\x94}\x94(\x8c\x05level\x94K\x00\x8c\x05lives\x94K\x04\x8c\x06points\x94Kdu\x85\x94R\x94.'


In [32]:
copyreg.pickle(BetterGameState, pickle_game_state)

state = BetterGameState()
serialized = pickle.dumps(state)
print(serialized)

b'\x80\x04\x95W\x00\x00\x00\x00\x00\x00\x00\x8c\x08__main__\x94\x8c\x13unpickle_game_state\x94\x93\x94}\x94(\x8c\x05level\x94K\x00\x8c\x06points\x94K\x00\x8c\x05magic\x94K\x05\x8c\x07version\x94K\x02u\x85\x94R\x94.'
